## Check new pipeline


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
from tqdm import tqdm
from cardio.dataset import SplitType
from cardio.configuration import *
from torch_geometric.loader import DataLoader
from cardio.dataset import Fame2RawDSLoader
from cardio.dataset.preprocess.graph import *
%load_ext autoreload
%autoreload 2

fame2_ds = Fame2RawDSLoader(
    
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    generate_data_with_2_views=True,
    duplicate_imgs_with_multiple_lesions=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup()


In [ ]:
ids = fame2_ds.df.patient_id_x

In [ ]:
fame2_ds = Fame2RawDSLoader(
    
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    keep_2views_data_for_comparison=True,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    duplicate_imgs_with_multiple_lesions=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup()


In [ ]:
pid_x = fame2_ds.df.patient_id_x

In [ ]:
import igraph as ig

def create_graph(coordinates, edge_index):
    g = ig.Graph(len(coordinates))
    g.vs['x'] = coordinates[:,0].tolist()
    g.vs['y'] = coordinates[:,1].tolist()

    coords = g.layout_auto().coords
    for edge in edge_index:
        g.add_edge(edge[0], edge[1])
    g.simplify() 

    return g

* Plot graph generated from pipeline

In [ ]:
import matplotlib.pyplot as plt

# Create pipeline
pipeline = ImageToGraphPipeline(
    SimplePixelExtractor(True, False, ['FFR', 'exactSegmentLocation'], [None, artery_segment_to_float]), 
    StochasticDownsampler(True), 
    KNNVertexConnector(5),
    # DelaunayVertexConnector(10),
    TASK_TO_LABEL_EXTRACTOR['EventForecast'],
    0.5, 1
)

ds_item = fame2_ds[100]

coordinates, features, _, _ = pipeline.generate_sampled_vertices(ds_item)
edge_index = pipeline.vertex_connector.connect(coordinates, features)
print(features.shape)

f, ax = plt.subplots(1,4, figsize=(20,5))
ax[0].scatter(coordinates[:,0], coordinates[:,1])


g = create_graph(coordinates, edge_index)
ig.plot(
    g,
    target=ax[1],
    vertex_size=0.04,
    vertex_color="lightblue",
    edge_width=0.8
)

ax[2].imshow(ds_item[0][0])
ax[3].imshow(ds_item[1])
print("Label:", pipeline.point_label_extractor_callback(ds_item[2]))

In [ ]:
from cardio.dataset import Fame2GraphDatasetWrapper
from torch_geometric.loader import DataLoader

from cardio.networks.gnn import *
%load_ext autoreload
%autoreload 2

graphDsWrapper = Fame2GraphDatasetWrapper()
ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root="data/datasets/eventForecastingKnnOnlyLesionsWFFR/train")
dl = DataLoader(ds, batch_size = 2)

it = iter(dl)
batch = next(it)

# x = torch.zeros((batch.x.shape[0], 256))
# aux_feats = batch.x[:, 3:]
# batch_index = batch.batch
# batch.y

In [ ]:
import torch
torch.vstack([g.lesion_wide_feat_tensor for g in ds]).std()

# Random classifiers predictions

Predictions for Event Forecasting task

In [ ]:
from cardio.dataset import Fame2GraphDatasetWrapper
from torch_geometric.loader import DataLoader
from sklearn.model_selection import KFold, train_test_split
import numpy as np
import torchmetrics.classification as metrics
from sklearn.metrics import confusion_matrix


# from cardio.networks.gnn import *
%load_ext autoreload
%autoreload 2

graphDsWrapper = Fame2GraphDatasetWrapper()
test_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root="data/datasets/eventForecastDelaunayOnlyLesions/test")


# Create a random test prediction using the train set proba distr
train_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root="data/datasets/eventForecastDelaunayOnlyLesions/train")
train_counts = train_ds.data.y.unique(return_counts=True)
classes, probas = train_counts[0], train_counts[1] / train_counts[1].sum()

test_metrics = [metrics.BinaryAccuracy(), metrics.BinaryPrecision(), metrics.BinaryRecall(), metrics.BinaryF1Score(), metrics.Specificity(), 
    metrics.AUROC()]

* Uniformly 1 and 0 

In [ ]:
import numpy as np
import torch

# Plot all scores
runs = 10
metrics_runs = {f"{m.__class__.__name__}":list() for m in test_metrics }
for r in range(runs):
    for m in test_metrics:
        pred = torch.rand(test_ds.data.y.shape)
        metrics_runs[f"{m.__class__.__name__}"].append(m(pred, test_ds.data.y.to(torch.long)).item())
        m.reset()

for c,v in metrics_runs.items():
    print(f"{c} = {np.mean(v):.2f} (+- {np.std(v):.2f})")

* Based on train label distribution 

In [ ]:
import numpy as np
import torch

runs = 10
metrics_runs = {f"{m.__class__.__name__}":list() for m in test_metrics }
for r in range(runs):
    for m in test_metrics:
        pred = torch.from_numpy(np.random.choice(classes, size=test_ds.data.y.shape, p=probas.numpy()))
        metrics_runs[f"{m.__class__.__name__}"].append(m(pred.to(torch.float), test_ds.data.y.to(torch.long)).item())  
        m.reset()

for c,v in metrics_runs.items():
    print(f"{c} = {np.mean(v):.2f} (+- {np.std(v):.2f})")  

Predictions for Lesion Detection task

In [ ]:
import cardio.pylightning as plmods

train_dataset_dir = "data/datasets/lesionDetectionDelaunay/train"
test_dataset_dir = "data/datasets/lesionDetectionDelaunay/test"

graphDsWrapper = Fame2GraphDatasetWrapper()
test_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=test_dataset_dir)


# Create a random test prediction using the train set proba distr
train_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=train_dataset_dir)
train_counts = train_ds.data.y.unique(return_counts=True)
classes, probas = train_counts[0], train_counts[1] / train_counts[1].sum()

test_metrics = [metrics.BinaryAccuracy(), metrics.BinaryPrecision(), metrics.BinaryRecall(), metrics.BinaryF1Score(), 
    metrics.AUROC()]

In [ ]:
import torchmetrics as metrics
import numpy as np
import torch

runs = 10
metrics_runs = {f"{m.__class__.__name__}":list() for m in test_metrics }
for r in range(runs):
    for m in test_metrics:
        pred = torch.rand(test_ds.data.y.shape)
        metrics_runs[f"{m.__class__.__name__}"].append(m(pred,test_ds.data.y.to(torch.long)).item())
        m.reset()

for c,v in metrics_runs.items():
    print(f"{c} = {np.mean(v):.2f} (+- {np.std(v):.2f})")

In [ ]:
import numpy as np
import torch


runs = 10
metrics_runs = {f"{m.__class__.__name__}":list() for m in test_metrics }
for r in range(runs):
    for m in test_metrics:
        pred = torch.from_numpy(np.random.choice(classes, size=test_ds.data.y.shape, p=probas.numpy()))
        metrics_runs[f"{m.__class__.__name__}"].append(m(pred.to(torch.float),test_ds.data.y.to(torch.long)).item())  
        m.reset()

for c,v in metrics_runs.items():
    print(f"{c} = {np.mean(v):.2f} (+- {np.std(v):.2f})")

# Predict using only FFR

* Use ROC curves on KFOLD to find best ffr threshold

In [ ]:
import cardio.resolvers as resolvers
import torch
import numpy as np

class Conf:
    gradient_clipping = None
    nb_epochs = 5
    eval_every_epochs = 1
    log_every_steps = 50
    val_batch_size = 8
    train_batch_size = 8
    num_workers = 4
    task = "forecast"
    checkpoint = False
    device = "cpu"
    model = "gnn"
    run_mode = "simple"
    dataset_dir = "data/datasets/eventForecastWithDiamSten"

confs = Conf()

''' Custom kflod fit using pyotrchlighting that returns validation metrics'''
folds = 5
confs.checkpoint = False # avoit checkpoints on kfold so we dont explode disk

# Resolvers
datamodule = resolvers.datamodule_resolver(confs)

# setup
datamodule.setup(None)


# datamodule.setup_folds(folds)
# Loop for all folds
# fold_predictions = [] 
# for fold in range(folds):
#     print(f"\n\n~~~~~ FOLD {fold} ~~~~~")
#     datamodule.setup_fold_index(fold)

#     val_y = list() 
#     val_ffr = list() 
#     for graph in datamodule.val_fold:
#         val_ffr.append( graph.x[:,3][graph.x[:,3] != 0.0][1].item() )
#         val_y.append( graph.y.item() )

#     fold_predictions.append( (np.array(val_y), np.array(val_ffr)) ) 

In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt


fprs, tprs, folds = [], [], []
for f, pred in enumerate(fold_predictions):

    y = pred[0]
    scores = 1-pred[1]
    fpr, tpr, thresholds = roc_curve(y, scores, pos_label=1.0) # high FFR means 0 not 1
    
    assert len(fpr) == len(tpr)
    for i in range(len(fpr)):
        fprs.append(fpr[i])
        tprs.append(tpr[i])
        folds.append(f)

    plt.plot(fpr, tpr)

plt.plot([0.0, 1.0], [0.0, 1.0], "--r")
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.title("KFold ROC Curve")
plt.show()


In [ ]:
plt.scatter(fpr, tpr, c=thresholds)
plt.plot([0.0, 1.0], [0.0, 1.0], "--r")
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.title("ROC curve for `1-FFR` threshold")

ax = plt.gca()
for i, txt in enumerate(thresholds):
    ax.annotate(f"{txt:.2f}", (fpr[i], tpr[i]))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

f, ax = plt.subplots(2,1, figsize=(5, 10))

for i, t in enumerate([0.20, 0.30]):
    ffr_threshold = t
    pred = []
    test_y = []
    for graph in datamodule.test_dataset:
        ffr = graph.x[:,3][graph.x[:,3] != 0.0][1].item()
        pred.append(1.0 if 1-ffr >= ffr_threshold else 0.0)
        test_y.append(graph.y.item())

    cm = confusion_matrix(test_y, pred)
    d =ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1]) 
    ax[i].set_title(f"CM for FFR {1-t}")
    d.plot(ax=ax[i])

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

f, ax = plt.subplots(1,1, figsize=(5, 5))

ds_threshold = 70
pred = []
test_y = []
for graph in datamodule.test_dataset:
    ds = graph.x[:,3][graph.x[:,3] != 0.0][1].item()
    pred.append(1.0 if ds >= ds_threshold else 0.0)
    test_y.append(graph.y.item())

cm = confusion_matrix(test_y, pred)
d =ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1]) 
ax.set_title(f"CM for Diameter Stenosis Prediction")
d.plot(ax=ax)

In [ ]:
import seaborn as sns
ffrs = np.concatenate([t[1] for t in fold_predictions])
labels = np.concatenate([t[0] for t in fold_predictions])

f, ax = plt.subplots(1)
sns.histplot(x=ffrs, hue=labels, ax= ax, bins=40)
ax.axvline(x=0.8, c="r")
ax.axvline(x=0.7, c="r")
ax.set_title("FFR distribution (Hue: events per ffr bar)")


NOTE: 
We can see that the best threshold is closer to `0.8` for the data.


* Lets predict the test set by using `0.8` as a threshold and plot all measures

In [ ]:
import torchmetrics.classification as metrics
import torchmetrics as tmetrics

ffr_threshold = 70
pred = []
test_y = []
for graph in datamodule.test_dataset:
    ffr = graph.x[:,3][graph.x[:,3] != 0.0][1].item()
    pred.append(1.0 if ffr >= ffr_threshold else 0.0)
    test_y.append(graph.y.item())

test_y = torch.tensor(test_y, dtype=torch.long)
pred = torch.tensor(pred, dtype=torch.float)

test_metrics = [metrics.BinaryAccuracy(), metrics.BinaryPrecision(), metrics.BinaryRecall(), metrics.BinaryF1Score(), tmetrics.AUROC(2,1), metrics.Specificity()]
metrics_runs = {f"{m.__class__.__name__}":list() for m in test_metrics }
for m in test_metrics:
    metrics_runs[f"{m.__class__.__name__}"].append(m(pred, test_y).item())  
    m.reset()

for c,v in metrics_runs.items():
    print(f"{c} = {np.mean(v):.2f}")

# KFOLD plots to compare lesion detection w FFR and w/out FFR models


In [ ]:
import pickle

with open('data/logs_lesion_wout_ffr.pickle', 'rb') as handle:
    lesion_wout_ffr = pickle.load(handle)

with open('data/logs_lesion_w_ffr.pickle', 'rb') as handle:
    lesion_with_ffr = pickle.load(handle)

lesion_wout_ffr.keys()


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

def compare_model_folds(metric_name, m1_metrics_dict, m2_metrics_dict, m1_name, m2_name, figsize = (10, 5)):
    # Model 1 metrics
    metric_m1, folds_m1 = zip(*m1_metrics_dict[metric_name])
    metric_m1, folds_m1 = [i.item() if isinstance(i, torch.Tensor) else i for i in metric_m1], list(folds_m1)
    
    cnts_m1 = Counter(folds_m1)
    assert len(set(cnts_m1.values())) == 1, "Folds of m1 have different lenghts"

    # Model 2 losses
    metric_m2, folds_m2 = zip(*m2_metrics_dict[metric_name])
    metric_m2, folds_m2 = [i.item() if isinstance(i, torch.Tensor) else i for i in metric_m2], list(folds_m2)

    cnts_m2 = Counter(folds_m2)
    assert len(set(cnts_m2.values())) == 1, "Folds of m2 have different lenghts"

  
    # Plot metric
    steps_m1 = [i for _ in range(0,len(cnts_m1)) for i in range(0,cnts_m1['fold-0'])]
    steps_m2 = [i for _ in range(0,len(cnts_m2)) for i in range(0,cnts_m2['fold-0'])]
    names_m1 =  [m1_name] * len(metric_m1)
    names_m2 =  [m2_name] * len(metric_m2)
    df = pd.DataFrame(data={
        metric_name: metric_m1 + metric_m2, 
        'Folds': folds_m1 + folds_m2, 
        'Model': names_m1 + names_m2})
    
    f, ax = plt.subplots(1,1, figsize=figsize)
    if metric_name == "val/epoch_loss":
        ax.set(yscale="log")
    sns.lineplot(data=df, y=metric_name, x= steps_m1 + steps_m2, hue="Model", ax=ax)
    ax.set_title(f"'{m1_name}' VS '{m2_name}' on metric: '{metric_name}'")


m1_name = "GNN w/out ffr"
m2_name = "GNN w/ ffr"

In [ ]:
# Plot train loss
metric_name = 'train/epoch_loss'
compare_model_folds(metric_name, lesion_wout_ffr, lesion_with_ffr, m1_name, m2_name)

In [ ]:
# Plot Accuracy
metric_name = 'val/epoch_Recall'
compare_model_folds(metric_name, lesion_wout_ffr, lesion_with_ffr, m1_name, m2_name)

In [ ]:
# Plot F1
metric_name = 'val/epoch_F1Score'
compare_model_folds(metric_name, lesion_wout_ffr, lesion_with_ffr, m1_name, m2_name)

In [ ]:
import pickle
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

with open('logs_storage.pickle', 'rb') as handle:
    model_logs = pickle.load(handle)

def plot_model_kfold(metric_name, m1_metrics_dict, m1_name, figsize = (10, 5)):
    # Model 1 metrics
    metric_m1, folds_m1 = zip(*m1_metrics_dict[metric_name])
    metric_m1, folds_m1 = [i.item() if isinstance(i, torch.Tensor) else i for i in metric_m1], list(folds_m1)
    
    cnts_m1 = Counter(folds_m1)
    assert len(set(cnts_m1.values())) == 1, "Folds of m1 have different lenghts"
  
    # Plot metric
    steps_m1 = [i for _ in range(0,len(cnts_m1)) for i in range(0,cnts_m1['fold-0'])]
    names_m1 =  [m1_name] * len(metric_m1)
    df = pd.DataFrame(data={
        metric_name: metric_m1, 
        'Folds': folds_m1, 
        'Model': names_m1})
    
    f, ax = plt.subplots(1,1, figsize=figsize)
    if metric_name == "val/epoch_loss":
        ax.set(yscale="log")
    sns.lineplot(data=df, y=metric_name, x= steps_m1, hue="Model", ax=ax)
    ax.set_title(f"'{m1_name}' on metric: '{metric_name}'")

plot_model_kfold("val/epoch_F1Score", model_logs, "name")

# Make FFR a probability distribution to predict Events

In [ ]:
import pandas as pd

# Load the data
df_cl = pd.read_csv('data/fame2_clinical_events_2year_data.csv')

# Useful columns
event_columns = ['VOCE', 'UR_TVF', 'NUR_TVF', 'MI_TVF', 'CV_TVF']
clinical_data_patient_lvl = ['CP_CN_MP_MN', 'HTN', 'Hchol', 'DM_Overall',
       'DM_Insulin', 'Ren_Ins', 'PVD', 'CVA', 'Prev_MI', 'Prev_PCI', 'CCS',
       'CCS_3', 'Silent_Ischemia', 'LVEF', 'MVD', 'Lesion_Type', 'Age', 'Male',
       'BMI', 'CAD', 'Smoker', 'Age_64', 'BMI_28']

# 
# Lesion level dataset
# 
df_cl.Prev_MI.fillna(0, inplace=True)
df_cl.Lesion_Length.fillna(0, inplace=True)
df_cl['Event'] = (df_cl.filter(event_columns).sum(axis=1) > 0).astype(int)

def sigmoid(x):
       return 1 / (1 + np.exp(-x))
df_cl['FFR2Proba'] = df_cl.FFR80.apply(lambda x: sigmoid(0.8 - x)) # FFR > 0.8 -> class 0 so we need to invert (-x)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import torch

f, ax = plt.subplots(1,3, figsize=(15,5))
sns.histplot(df_cl.FFR , ax=ax[0])
sns.histplot(0.8 - df_cl.FFR, ax=ax[1])

ffr = torch.tensor(df_cl.FFR)
ffr_sm = torch.nn.functional.sigmoid(0.8 - ffr)
sns.histplot(ffr_sm, ax=ax[2])

In [ ]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score
from torchmetrics import AUROC, Precision
import numpy as np

y_true = torch.tensor(df_cl.Event)
pred = ffr_sm

test_metrics = [BinaryAccuracy(), BinaryPrecision(), BinaryRecall(), BinaryF1Score(), AUROC(2,1)]
metrics_runs = {f"{m.__class__.__name__}":list() for m in test_metrics }
for m in test_metrics:
    metrics_runs[f"{m.__class__.__name__}"].append(m(pred, y_true).item())  
    m.reset()

for c,v in metrics_runs.items():
    print(f"{c} = {np.mean(v):.2f}")

In [ ]:
y_true = torch.tensor(df_cl.Event)
pred = torch.tensor(df_cl.FFR.apply(lambda x: 0.0 if x > 0.8 else 1.0))

test_metrics = [BinaryAccuracy(), BinaryPrecision(), BinaryRecall(), BinaryF1Score(), AUROC(2,1)]
metrics_runs = {f"{m.__class__.__name__}":list() for m in test_metrics }
for m in test_metrics:
    metrics_runs[f"{m.__class__.__name__}"].append(m(pred, y_true).item())  
    m.reset()

for c,v in metrics_runs.items():
    print(f"{c} = {np.mean(v):.2f}")

# T-SNE on node embeddings  

In [ ]:
# Load the data
from cardio.dataset import Fame2GraphDatasetWrapper
import cardio.resolvers as resolvers
import torch
import numpy as np

CHECKPOINT_PATH_GNN = "checkpoint/GIN-L1-F3-E128_LesionPooling-G512_E20_EF_DE.ckpt"

class Conf:
    gradient_clipping = None
    nb_epochs = 5
    eval_every_epochs = 1
    log_every_steps = 50
    val_batch_size = 1 
    train_batch_size = 1 
    test_batch_size = 1 
    num_workers = 4
    checkpoint = False
    device = "cpu"
    model = "gnn"
    run_mode = "simple"
    train_dataset_dir = "data/datasets/eventForcastingDelaunay/train"
    test_dataset_dir = "data/datasets/eventForcastingDelaunay/test"

    # Model configs
    gnn_norm = "BatchNorm"
    gnn_pooling_cls_name = "LesionPooling"
    weight_init_strategy = ""
    gnn_cls_name = "GIN"
    gnn_nb_layers = 1
    gnn_dropout = 0.5973039417606918     
    gnn_node_emb_dim = 128
    gnn_global_hidden_dim = 512
    gnn_ffr_pooling_proj_dim = 16
    gnn_node_features = 3
    gnn_ffr_pooling_factor = 2
    gnn_act = 'relu'
    nb_classes = 1
    weight_init_strategy = "Xavier Uniform"
    init_weights_from = None
    gnn_freeze_weights = False

    init_weights_for = 'both'
    init_weights_from= CHECKPOINT_PATH_GNN
    use_balanced_batches = False
    
confs = Conf()

''' Custom kflod fit using pyotrchlighting that returns validation metrics'''
folds = 5
confs.checkpoint = False # avoit checkpoints on kfold so we dont explode disk

# Resolvers
graphDsWrapper = Fame2GraphDatasetWrapper()
train_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=confs.train_dataset_dir)
# test_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=confs.test_dataset_dir )
model = resolvers.model_resolver(confs, None, None, None).model
model.eval()
print(train_ds)

* Plot with T-SNE all the nodes in the train set

In [ ]:
node_vecs = np.zeros((len(train_ds), 128), dtype=np.float32)
y_train = np.zeros((len(train_ds), 1), dtype=np.int32)
ffr_train = np.zeros((len(train_ds), 1), dtype=np.float32)

def model_predict(model, input):
    return torch.sigmoid(model(input)).item()

# Create Train 
for i, graph in enumerate(train_ds):
    # Prepare data
    ffr = graph.x[:,3][graph.x[:,3] != 0.0][1].item() # FFR is in the last column of x, but only exists for lesion nodes

    y_train[i] = graph.y.item()
    ffr_train[i,0] = ffr

    batched_x = graph.x[:, :3]
    node_vecs[i, :] = model.gnn(batched_x, graph.edge_index).detach().numpy().mean(axis=0).shape

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 

nodes_tsne = TSNE(n_components=2, learning_rate='auto',
    init='random', perplexity=3).fit_transform(node_vecs)

plt.scatter(nodes_tsne[:,0], nodes_tsne[:,1], c=y_train)

* Plot multiple graphs on T-SNE to see if lesion pools are are different from non-lesion ones

In [ ]:
batched_data = train_ds[700]

batched_x = batched_data.x[:, :3]
ffr_feats = batched_data.x[:,3:]
ffr_feats[ffr_feats != 0.] = 1 
out = model.gnn(batched_x, batched_data.edge_index).detach().numpy()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 

X_embedded = TSNE(n_components=2, learning_rate='auto',
    init='random', perplexity=3).fit_transform(out)

In [ ]:
plt.scatter(X_embedded[:, 0], X_embedded[:,1], c=ffr_feats)

In [ ]:
plt.scatter(X_embedded[:, 0], X_embedded[:,1], c=ffr_feats)

In [ ]:
plt.scatter(X_embedded[:, 0], X_embedded[:,1], c=ffr_feats)

In [ ]:
plt.scatter(X_embedded[:, 0], X_embedded[:,1], c=ffr_feats)

# Predict with pre-trained models

In [ ]:
# Load the data
import cardio.resolvers as resolvers
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from cardio.dataset import Fame2GraphDatasetWrapper
import numpy as np
from tqdm import tqdm

CHECKPOINT_PATH_GNN = "checkpoint/diameterStenosisClf.ckpt"

class Conf:
    gradient_clipping = None
    nb_epochs = 5
    eval_every_epochs = 1
    log_every_steps = 50
    val_batch_size = 2 
    train_batch_size = 2 
    test_batch_size = 2
    num_workers = 1
    checkpoint = False
    device = "cpu"
    model = "gnn"
    run_mode = "simple"
    task = "forecast"
    dataset_dir = "data/datasets/eventForecastingDelaunayWithClinicalData"
    
    # Model configs
    gnn_norm = "BatchNorm"
    gnn_pooling_cls_name = "CustomPooling"  
    weight_init_strategy = ""
    gnn_cls_name = "GIN"
    gnn_nb_layers = 5
    gnn_dropout = 0.5973039417606918     
    gnn_node_emb_dim = 256
    gnn_global_hidden_dim = 512
    
    gnn_ffr_pooling_proj_dim = 16
    gnn_node_features = 4
    gnn_ffr_pooling_factor = 2

    gnn_act = 'relu'
    nb_classes = 1
    weight_init_strategy = "Xavier Uniform"
    init_weights_from = None
    gnn_freeze_weights = False
    loss_func="FocalLoss"
    checkpoint_after_n_epochs = 5

    init_weights_for = 'both'
    init_weights_from= CHECKPOINT_PATH_GNN
    use_balanced_batches = True
    
confs = Conf()

''' Custom kflod fit using pyotrchlighting that returns validation metrics'''
folds = 5
confs.checkpoint = False # avoit checkpoints on kfold so we dont explode disk

# Resolvers
graphDsWrapper = Fame2GraphDatasetWrapper()
# train_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=confs.train_dataset_dir)
# test_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=confs.test_dataset_dir )
dm = resolvers.datamodule_resolver(confs)
model = resolvers.model_resolver(confs, None, None, None).model
model.eval()

dm.setup(None)

In [ ]:
from sklearn.model_selection import train_test_split

# Go from graphs to patients
pat_id = pd.DataFrame(data=[(g.patient_id, g.y.item()) for g in train_ds], columns=["patient", "voce"])
grouped_patients = pat_id.groupby("patient").mean() > 0.1
y = grouped_patients['voce'].values
pat_ids = grouped_patients.index.values

train_pat_ids, val_patient_ids = train_test_split(np.arange(len(y)), test_size=0.1, stratify=y)

# Go back to patients from graphs
graph_ds_indexes_train = [i for i, g in enumerate(train_ds) if g.patient_id in pat_ids[train_pat_ids]] 
graph_ds_indexes_val = [i for i, g in enumerate(train_ds) if g.patient_id in pat_ids[val_patient_ids]] 
len(graph_ds_indexes_train), len(graph_ds_indexes_val)

In [ ]:
train_ds.index_select(graph_ds_indexes_val)

In [ ]:
from collections import Counter
print(Counter(grouped_patients.iloc[x2]['voce'].values.astype(np.int32)))
print(Counter(grouped_patients.iloc[x]['voce'].values.astype(np.int32)))
print(Counter(y))

In [ ]:
pred_train = np.zeros((len(train_ds), 1), dtype=np.float32)
y_train = np.zeros((len(train_ds), 1), dtype=np.int32)

pred_test = np.zeros((len(test_ds), 1), dtype=np.float32)
y_test = np.zeros((len(test_ds), 1), dtype=np.int32)

# Create Train 
for i, graph in tqdm(enumerate(train_ds), desc="train data", total=len(train_ds)):
    # Prepare data
    graph.batch = torch.zeros(len(graph.x), dtype=torch.int64)

    y_train[i] = graph.y.item()
    pred_train[i] = model(graph).detach().numpy()

# Create Test
for i, graph in tqdm(enumerate(test_ds), desc="test data", total=len(test_ds)):
    
    # Prepare data
    graph.batch = torch.zeros(len(graph.x), dtype=torch.int64)

    y_test[i] = graph.y.item()
    pred_test[i] = model(graph).detach().numpy()

In [ ]:
f, ax = plt.subplots(1,2)
sig_pred_train = 1/(1 + np.exp(-pred_train.reshape(-1)))
sns.histplot(x=sig_pred_train > 0.5, ax=ax[0])
sns.histplot(x=y_train.reshape(-1), ax=ax[1])


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


sig_pred_test = 1/(1 + np.exp(-pred_test.reshape(-1)))
cm = confusion_matrix(y_test, sig_pred_test > 0.5)
cmd = ConfusionMatrixDisplay(cm)
cmd.plot()

In [ ]:
from collections import Counter
Counter(y_test.reshape(-1))

In [ ]:
import pandas as pd
df = pd.read_csv("data/fame2_clinical_events_2year_data.csv")
test_pats = pd.read_csv("data/datasets/test_split.csv", header=None)
l = [f[0] for f in test_pats.values]

df_test = df[df.Patient.isin(l)]
df_test.DS.hist()
plt.axvline(x=50, c="r")



# Centerline

In [ ]:
import pandas as pd

from cardio.dataset import SplitType
from cardio.configuration import *
from torch_geometric.loader import DataLoader
from cardio.dataset import Fame2RawDSLoader

fame2_ds = Fame2RawDSLoader(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    duplicate_imgs_with_multiple_lesions=False, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup(split_path=SPLIT_PATH,
    split_type= SplitType.train)

In [ ]:
fame2_ds.df

In [ ]:
l = []
for a, d in fame2_ds.df.groupby(['base_path', 'artery', 'lesion_name']):
    l.append( d.has_lesion.values[0] )

Counter(l)


In [ ]:
d.has_

In [ ]:
from skimage.morphology import skeletonize
from skimage import data
import matplotlib.pyplot as plt
import cv2

# Invert the horse image
# image = invert(data.horse())
image = fame2_ds[0][0][1,:].numpy()
# perform skeletonization

# may need to close holes
kernelSize = (3,3)  
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
image_filled = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
skeleton = skeletonize(image_filled)

# display results
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 4),
                         sharex=True, sharey=True)

ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].axis('off')
ax[0].set_title('original', fontsize=20)

ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].axis('off')
ax[1].set_title('skeleton', fontsize=20)

ax[2].imshow(image + skeleton , cmap=plt.cm.gray)
ax[2].axis('off')
ax[2].set_title('image + skeleton', fontsize=20)

fig.tight_layout()
plt.show()

# Find % of same images we classify differently because they are different views


In [ ]:
import pandas as pd

from cardio.dataset import SplitType
from cardio.configuration import *
from cardio.dataset import Fame2RawDSLoader

fame2_ds = Fame2RawDSLoader(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    duplicate_imgs_with_multiple_lesions=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup(split_path=SPLIT_PATH,
    split_type= SplitType.train)

In [ ]:
import cv2

raw_stacked, lesion_mask, df_info = fame2_ds[0]

raw_img = raw_stacked[0].numpy().astype(np.uint8)
artery_mask = raw_stacked[1].numpy().astype(np.uint8)
lesion_mask = lesion_mask.numpy().astype(np.uint8)

edge_mask = cv2.Canny(artery_mask, 1, 2)
coordinates = np.vstack(np.where(edge_mask != 0)).transpose(1,0)
assert len(np.unique(lesion_mask)) == 3 and df_info.shape[0] == 1
lesion_points = (lesion_mask[coordinates[:,0], coordinates[:,1]] == 2).astype(np.float32).reshape(-1,1)

attach_features = ['FFR']
feature_transforms = [ None ]
feature_tensors = []
for i in range(len(attach_features)):
    feature = attach_features[i]
    transform = feature_transforms[i]
    if transform != None:
        value = transform(df_info[feature].values[0])
    else:
        value = df_info[feature].values[0]
    assert not pd.isna(value), f"Could not attach value to data point, due to null feature `{feature}`"

    # An array where we have only N elements (bases on teh edge_mask coordiantes) and the values are 
    # 0 if not a lesion and 1 if a lesion
    feature = (lesion_mask[coordinates[:,0], coordinates[:,1]] == 2).astype(np.float32).reshape(-1,1)
    # Then use the lesion only points of the edge and assign them the value
    feature[feature == 1.0] = value
    feature_tensors.append(feature)

# Skip the pixel intensity
features = np.hstack([raw_img[coordinates[:,0], coordinates[:,1]].reshape(-1,1), coordinates] + feature_tensors)

coordinates.shape, features.shape, edge_mask.shape, lesion_points.shape

In [ ]:
# Load the data
import cardio.resolvers as resolvers
import torch
import numpy as np
from cardio.dataset import  Fame2GraphDatasetWrapper

CHECKPOINT_PATH_GNN = "checkpoint/GIN-L1-F3-E128_LesionPooling-G512_E20_EF_DE.ckpt"

class Conf:
    loss_func = "BCEWithLogitsLoss"
    task = "forecast"
    gradient_clipping = None
    nb_epochs = 5
    eval_every_epochs = 1
    log_every_steps = 50
    val_batch_size = 1 
    train_batch_size = 1 
    test_batch_size = 1 
    num_workers = 4
    checkpoint = False
    device = "cpu"
    model = "gnn"
    run_mode = "simple"
    train_dataset_dir = "/Users/tbelmpas/Code/EPFL/thesis/TheoBelmpas-2022-mas-MiFromFAME2-AI4CARDIO/data/datasets/eventForecastingDelaunayNoPixelsOnlyLesions/train"
    test_dataset_dir = "/Users/tbelmpas/Code/EPFL/thesis/TheoBelmpas-2022-mas-MiFromFAME2-AI4CARDIO/data/datasets/eventForecastingDelaunayNoPixelsOnlyLesions/test"

    # Model configs
    gnn_norm = "BatchNorm"
    gnn_pooling_cls_name = "LesionPooling"
    weight_init_strategy = ""
    gnn_cls_name = "GIN"
    gnn_nb_layers = 1
    gnn_dropout = 0.5973039417606918     
    gnn_node_emb_dim = 128
    gnn_global_hidden_dim = 512
    gnn_ffr_pooling_proj_dim = 16
    gnn_node_features = 3
    gnn_ffr_pooling_factor = 2
    gnn_act = 'relu'
    nb_classes = 1
    weight_init_strategy = "Xavier Uniform"
    init_weights_from = None
    gnn_freeze_weights = False

    init_weights_for = 'both'
    init_weights_from= CHECKPOINT_PATH_GNN
    use_balanced_batches = False
    
import os
confs = Conf()
if not os.path.isdir(confs.train_dataset_dir):
    raise ValueError("Train path doesnt exist")

''' Custom kflod fit using pyotrchlighting that returns validation metrics'''
folds = 5
confs.checkpoint = False # avoit checkpoints on kfold so we dont explode disk

# Resolvers
graphDsWrapper = Fame2GraphDatasetWrapper()
train_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=confs.train_dataset_dir)
test_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=confs.test_dataset_dir )
model = resolvers.model_resolver(confs, None).model
model.eval()

print(train_ds)

In [ ]:
from tqdm import tqdm

y_train = np.zeros((len(train_ds), 1), dtype=np.int32)
pred_train = np.zeros((len(train_ds), 1), dtype=np.float32)
ffr_train = np.zeros((len(train_ds), 1), dtype=np.float32)
train_lesions_id = [None] * len(train_ds)

y_test = np.zeros((len(test_ds), 1), dtype=np.int32)
pred_test = np.zeros((len(test_ds), 1), dtype=np.float32)
ffr_test = np.zeros((len(test_ds), 1), dtype=np.float32)
test_lesions_id = [None] * len(test_ds)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def model_predict(model, input):
    return torch.sigmoid(model(input)).item()

# Create Train 
fame2_ds.setup("data/datasets/", SplitType.train)
for i, graph in tqdm(enumerate(train_ds), total=len(train_ds)):
    # Prepare data
    ds_items = fame2_ds[i]
    graph.batch = torch.zeros(len(graph.x), dtype=torch.int64)
    ffr = graph.x[:,3][graph.x[:,3] != 0.0][1].item() # FFR is in the last column of x, but only exists for lesion nodes
    lesion_id = ds_items[2].apply(
            lambda x: f"{x['patient_id_x']}_{x['lesion_name'].replace('lesion', '') if x['lesion_name'] else 'None'}",
            axis =1).values[0]

    if (not np.isclose(ffr, ds_items[2].FFR.values[0])):
        print( f"Found diff ffrs in line {i} -> {ffr:.2f} vs {ds_items[2].FFR.values[0]:.2f}" )
        raise RuntimeError("bra")

    y_train[i] = graph.y.item()
    pred_train[i] = model_predict(model, graph)
    ffr_train[i,0] = ffr
    train_lesions_id[i] = lesion_id 

# Create Test
fame2_ds.setup("data/datasets/", SplitType.test)
for i, graph in tqdm(enumerate(test_ds), total=len(test_ds)):
    
    # Prepare data
    ds_items = fame2_ds[i]
    graph.batch = torch.zeros(len(graph.x), dtype=torch.int64)
    ffr = graph.x[:,3][graph.x[:,3] != 0.0][1].item() # FFR is in the last column of x, but only exists for lesion nodes
    lesion_id = ds_items[2].apply(
            lambda x: f"{x['patient_id_x']}_{x['lesion_name'].replace('lesion', '') if x['lesion_name'] else 'None'}",
            axis =1).values[0]

    assert np.isclose(ffr, ds_items[2].FFR.values[0]), f"Found diff ffrs in line {i}"

    y_test[i] = graph.y.item()
    pred_test[i] = model_predict(model, graph)
    ffr_test[i,0] = ffr
    test_lesions_id[i] = lesion_id 


In [ ]:
train_data = pd.DataFrame(data={
    'lesion_id': train_lesions_id, 'pred': pred_train.reshape(-1), 
    'label': y_train.reshape(-1), 'ffr': ffr_train.reshape(-1) 
})

test_data = pd.DataFrame(data={
    'lesion_id': test_lesions_id, 'pred': pred_test.reshape(-1), 
    'label': y_test.reshape(-1), 'ffr': ffr_test.reshape(-1) 
})

len(train_data.groupby('lesion_id').count())/len(train_data), len(test_data.groupby('lesion_id').count())/len(test_data)

In [ ]:
def most_confident(gdf):
    preds = gdf.pred.values - 0.5
    preds[preds <0.0] = preds[preds <0.0] * -1
    return pd.Series(data=[gdf.pred.values[np.argmax(preds)], gdf.label.values[0]], index=["pred", "label"])
most_conf_test_data = test_data.groupby('lesion_id').apply(most_confident)
most_conf_test_data.describe()


In [ ]:
import matplotlib.pyplot as plt
agg_test_data = test_data.groupby('lesion_id').agg(
    pred_mean=('pred', 'mean'), stdp=('pred', 'std'), 
    pred_min=('pred', 'min'), label=('label', 'min'),
    pred_max=('pred', 'max'))

def most_confident(gdf):
    preds = gdf.pred.values - 0.5
    preds[preds <0.0] = preds[preds <0.0] * -1
    return pd.Series(data=[gdf.pred.values[np.argmax(preds)], gdf.label.values[0]], index=["pred", "label"])
most_conf_test_data = test_data.groupby('lesion_id').apply(most_confident)

f, ax = plt.subplots(1,2)
agg_test_data.pred_mean.hist(ax=ax[0])
agg_test_data.stdp.hist(ax=ax[1])

In [ ]:
def f(gdf):
    
    return 
test_data.groupby('lesion_id')

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

metrics_no_agg = [
    accuracy_score(test_data.label.values, test_data.pred.values > 0.5), 
    recall_score(test_data.label.values, test_data.pred.values > 0.5),
    precision_score(test_data.label.values, test_data.pred.values > 0.5),
    f1_score(test_data.label.values, test_data.pred.values > 0.5),
    roc_auc_score(test_data.label.values, test_data.pred.values > 0.5)]

metrics_min = [
    accuracy_score(agg_test_data.label.values, agg_test_data.pred_min.values > 0.5), 
    recall_score(agg_test_data.label.values, agg_test_data.pred_min.values > 0.5),
    precision_score(agg_test_data.label.values, agg_test_data.pred_min.values > 0.5),
    f1_score(agg_test_data.label.values, agg_test_data.pred_min.values > 0.5),
    roc_auc_score(agg_test_data.label.values, agg_test_data.pred_min.values > 0.5)]

metrics_max = [
    accuracy_score(agg_test_data.label.values, agg_test_data.pred_max.values > 0.5), 
    recall_score(agg_test_data.label.values, agg_test_data.pred_max.values > 0.5),
    precision_score(agg_test_data.label.values, agg_test_data.pred_max.values > 0.5),
    f1_score(agg_test_data.label.values, agg_test_data.pred_max.values > 0.5),
    roc_auc_score(agg_test_data.label.values, agg_test_data.pred_max.values > 0.5)]

metrics_most_confident = [
    accuracy_score(most_conf_test_data.label.values, most_conf_test_data.pred.values > 0.5), 
    recall_score(most_conf_test_data.label.values, most_conf_test_data.pred.values > 0.5),
    precision_score(most_conf_test_data.label.values, most_conf_test_data.pred.values > 0.5),
    f1_score(most_conf_test_data.label.values, most_conf_test_data.pred.values > 0.5),
    roc_auc_score(most_conf_test_data.label.values, most_conf_test_data.pred.values > 0.5)]

metrics_df = pd.DataFrame(data={'no_agg': metrics_no_agg ,'use_min': metrics_min, 'use_max': metrics_max, 'use_most_conf': metrics_most_confident}, index=['Acc', 'Rcl', 'Prc', 'F1', 'Auc'])
metrics_df


In [ ]:
import pandas as pd

from cardio.dataset import SplitType
from cardio.configuration import *
from cardio.dataset import generate_fame2_patches_ds

generate_fame2_patches_ds(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    EVENT_COLUMNS, 
    "data/datasets/baselinePatches/",
    "data/datasets/"
)


In [ ]:
from cardio.pylightning import Fame2Baseline
from torchvision import transforms


preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456], std=[0.229, 0.224])
])

class Conf:
    dataset_dir:str
    num_workers = 4
    train_batch_size = 2
confs = Conf()
confs.dataset_dir = "data/datasets/baselinePatches"
dm = Fame2Baseline(confs, preprocess)
dm.setup(None)

In [ ]:
import cardio.networks.baseline as bs

dl = dm.train_dataloader()
i = next(iter(dl))
model = bs.BaselineResNet(0.3)

# Random Node removes to make graph denser

In [ ]:
import pandas as pd
from tqdm import tqdm
from cardio.dataset import SplitType
from cardio.configuration import *
from torch_geometric.loader import DataLoader
from cardio.dataset import Fame2RawDSLoader
from cardio.dataset.preprocess.graph import *
from skimage.morphology import skeletonize
%load_ext autoreload
%autoreload 2

fame2_ds = Fame2RawDSLoader(
    
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    duplicate_imgs_with_multiple_lesions=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup()


pipeline = ImageToGraphPipeline(
    LesionOnlyPixelExtractor(True, ['FFR', 'exactSegmentLocation'], [None, artery_segment_to_float]), 
    PixelSampler(), 
    DelaunayVertexConnector(10),
    TASK_TO_LABEL_EXTRACTOR['EventForecast'],
)

In [ ]:
fame2_ds.df.DS

In [ ]:
import matplotlib.pyplot as plt

pipeline = ImageToGraphPipeline(
    LesionOnlyPixelExtractor(True, ['FFR', 'exactSegmentLocation'], [None, artery_segment_to_float]), 
    NoSampler(), 
    DelaunayVertexConnector(10),
    TASK_TO_LABEL_EXTRACTOR['EventForecast'],
    1, 1
)
f, ax = plt.subplots(1,2, figsize=(20,5))

ds_item = fame2_ds[834]

coordinates, features, lesion_points = pipeline.generate_sampled_vertices(ds_item)
edge_index = pipeline.vertex_connector.connect(coordinates, features)
print(features.shape)
find_neighs_of = 5
al = torch.where( edge_index[:,0] == find_neighs_of )[0]
ar = torch.where( edge_index[:,1] == find_neighs_of )[0]
a = list(set( edge_index[al][:,1].numpy().tolist() + edge_index[ar][:,0].numpy().tolist()))
neighs = coordinates[a]


ax[0].scatter(coordinates[:,0], coordinates[:,1], s=1)
ax[0].scatter(neighs[:,0], neighs[:,1], s=6)


pipeline = ImageToGraphPipeline(
    LesionOnlyPixelExtractor(True, ['FFR', 'exactSegmentLocation'], [None, artery_segment_to_float]), 
    StochasticDownsampler(), 
    DelaunayVertexConnector(10),
    TASK_TO_LABEL_EXTRACTOR['EventForecast'],
    0.25,
    1
)
coordinates, features, lesion_points = pipeline.generate_sampled_vertices(ds_item)
edge_index = pipeline.vertex_connector.connect(coordinates, features)
print(features.shape)
al = torch.where( edge_index[:,0] == find_neighs_of )[0]
ar = torch.where( edge_index[:,1] == find_neighs_of )[0]
a = list(set( edge_index[al][:,1].numpy().tolist() + edge_index[ar][:,0].numpy().tolist()))
neighs = coordinates[a]

ax[1].scatter(coordinates[:,0], coordinates[:,1], s=1)
ax[1].scatter(neighs[:,0], neighs[:,1], s=7)

print("HERE", pipeline.point_label_extractor_callback(ds_item[2]))

In [ ]:
image = ds_item[0][1].numpy().astype(np.uint8)
kernelSize = (4,4)  
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
image_filled = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
skeleton = skeletonize(image_filled)
skeleton_coordinates_x = np.where(skeleton)[0]
skeleton_coordinates_y = np.where(skeleton)[1]
skeleton_coordinates = np.vstack([skeleton_coordinates_x, skeleton_coordinates_y])

plt.imshow(skeleton)

In [ ]:
min_distances = np.zeros((len(coordinates),1))
for i, point in enumerate(coordinates.numpy()):
    b = skeleton_coordinates - point.reshape(2,1)
    min_distances[i] = np.min(np.linalg.norm(b,axis=0))

min_distances.mean(), min_distances.std()


In [ ]:
plt.scatter(np.arange(min_distances.shape[0]), min_distances)

# Count data points with specific attributes

* count data points with ffr in between 0.7 and 0.8

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from cardio.dataset import SplitType
from cardio.configuration import *
from cardio.dataset import Fame2RawDSLoader
from cardio.dataset.fame2_raw_data_loader import load_train_test_split

fame2_ds = Fame2RawDSLoader(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    generate_data_with_2_views=True,
    duplicate_imgs_with_multiple_lesions=False,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup(SPLIT_PATH, SplitType.train)
train_patients, test_patients = load_train_test_split(SPLIT_PATH)


In [ ]:
df = fame2_ds.df
# group_columns =  ['patient_id_x', 'artery']
group_columns = ['base_path', 'artery']

l = list(df[(df.has_label) & (df.has_lesion)]
                    # .groupby(group_columns).filter(lambda x: len(x) == 2) # keep only 2 views
                    .groupby(group_columns))
len(l)

# Find differences in predictions between: FFR, DS and GNN

* Load the data

In [ ]:
import cardio.resolvers as resolvers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cardio.dataset import SplitType
from cardio.configuration import *
from cardio.dataset import Fame2RawDSLoader
from cardio.dataset.fame2_raw_data_loader import load_train_test_split

fame2_ds = Fame2RawDSLoader(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    generate_data_with_2_views=True,
    duplicate_imgs_with_multiple_lesions=False,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

class Conf:
    gradient_clipping = None
    nb_epochs = 5
    eval_every_epochs = 1
    log_every_steps = 50
    val_batch_size = 8
    train_batch_size = 8
    num_workers = 4
    task = "forecast"
    checkpoint = False
    device = "cpu"
    model = "gnn"
    run_mode = "simple"
    dataset_dir = "data/datasets/eventForecastDelaunayOnlyLesions"
confs = Conf()
confs.checkpoint = False # avoit checkpoints on kfold so we dont explode disk

# Resolvers
datamodule = resolvers.datamodule_resolver(confs)

# setup
datamodule.setup(None)

# get the dataframe with clinical data
df = fame2_ds.df

* Predict FFR

In [ ]:
import torchmetrics.classification as metrics
import torchmetrics as tmetrics

# 
ffr_threshold = 0.8
ds_threshold = 70
test_ffr, train_ffr = [], []
test_ds, train_ds = [], []
test_y, train_y = [], []
for type, ds in [("test", datamodule.test_dataset), ("train", datamodule.train_dataset)]:
    for graph in ds:
        ffr = df[(df.patient_id_x == graph.patient_id) & (df.lesion_id_syntax == graph.lesion_id)].FFR.values[0]
        ds = df[(df.patient_id_x == graph.patient_id) & (df.lesion_id_syntax == graph.lesion_id)].DS.values[0]
        assert not pd.isna(ds) and not pd.isna(ffr)
        
        if type == 'test':
            test_ffr.append(ffr)
            test_ds.append(ds)
            test_y.append(graph.y.item())
        else:
            train_ffr.append(ffr)
            train_ds.append(ds)
            train_y.append(graph.y.item())

test_ffr = np.array(test_ffr)
test_ds = np.array(test_ds)
test_y = torch.tensor(test_y, dtype=torch.long)
test_pred_ffr = torch.tensor([1.0 if ffr < ffr_threshold else 0.0 for ffr in test_ffr], dtype=torch.float)
test_pred_ds  = torch.tensor([1.0 if ds >= ds_threshold else 0.0 for ds in test_ds], dtype=torch.float)


train_ffr = np.array(train_ffr)
train_ds = np.array(train_ds)
train_y = torch.tensor(train_y, dtype=torch.long)
train_pred_ffr = torch.tensor([1.0 if ffr < ffr_threshold else 0.0 for ffr in train_ffr], dtype=torch.float)
train_pred_ds  = torch.tensor([1.0 if ds >= ds_threshold else 0.0 for ds in train_ds], dtype=torch.float)

test_metrics = [metrics.BinaryAccuracy(), metrics.BinaryPrecision(), metrics.BinaryRecall(), metrics.BinaryF1Score(), tmetrics.AUROC(2,1), metrics.Specificity(), tmetrics.ConfusionMatrix(2)]
metrics_store = { f"{m.__class__.__name__}" if m.__class__.__name__ != "ConfusionMatrix" else "tn, fp, fn,tp" :list()  for m in test_metrics }
metrics_store['type'] = list()
metrics_store['split'] = list()
for type, split, pred in [('ffr', 'test', test_pred_ffr), ('ds','test', test_pred_ds), ('ffr', 'train', train_pred_ffr), ('ds','train', train_pred_ds)]:
    y_true = test_y if split == 'test' else train_y
    for m in test_metrics:
        if m.__class__.__name__ == "ConfusionMatrix":
            cm  = m(pred, y_true)
            metrics_store["tn, fp, fn,tp"].append( cm.numpy().flatten() )
        else:
            metrics_store[f"{m.__class__.__name__}"].append(m(pred, y_true).item())  
        m.reset()
    metrics_store['type'].append(type)
    metrics_store['split'].append(split)

scores_ffr_ds = pd.DataFrame(data=metrics_store).set_index(["split", "type"])

scores_ffr_ds

* Plot the ffr and ds distr on test set

In [ ]:
f, ax = plt.subplots(2,2, figsize=(10,10))
ax[0][0].set_title("DS hist test")
ax[0][0].hist(test_ds)
ax[0][1].set_title("FFR hist test")
ax[0][1].hist(test_ffr, bins=30)

ax[1][0].set_title("DS hist train")
ax[1][0].hist(train_ds)
ax[1][1].set_title("FFR hist train")
ax[1][1].hist(train_ffr, bins=30)
f.tight_layout()

* Check if predictions of FFR and DS overlap in terms of fp and fn

In [ ]:
# Are FFR and DS predictions for events overlapping 
overlap_ffr_ds = torch.where( (test_pred_ds == 1) & (test_pred_ffr == 1) )[0]
overlap_ffr_ds_to_real = torch.where( test_y[overlap_ffr_ds] == 1)[0]
assert overlap_ffr_ds_to_real.shape[0] == 23

# Are the 14 fp of ds in the fp set of ffr also
fp_ds = torch.where( (test_pred_ds == 1) & (test_y == 0) )[0]
overlap_fp_ds_w_fp_ffr = torch.where( test_pred_ffr[fp_ds] == 1 )[0]
print(overlap_fp_ds_w_fp_ffr.shape[0], fp_ds.shape[0])

# ---> Yes FFR predicted events on every point DS predicted events too 

* Check what are the value distributions of fn for both FFR and DS

In [ ]:
ds_fn = torch.where((test_pred_ds == 0) & (test_y == 1))[0].numpy()
ffr_fn = torch.where((test_pred_ffr == 0) & (test_y == 1))[0].numpy()

f, ax = plt.subplots(1,2)
ax[0].set_title("DS false positive value distribution")
ax[0].hist(test_ds[ds_fn])
ax[1].set_title("FFR false positive value distribution")
ax[1].hist(test_ffr[ffr_fn])
f.tight_layout()
print(f"Len fn ds: {ds_fn.shape[0]} | Len fn ffr: {ffr_fn.shape[0]}")

* Docs said that all "unknown" ffr's take a default 0.5 value, let's see how many we miss with values == 0.5

**Answer:** This doesn't affect our predictions because only 10/111 false positives have a value of 0.5

**hint**: We can remedy the fp of FFR by testing if the DS is high enough
* To do that, we cannot use th DS, because the points in-between 0.7 and 0.8 (which case the fp to be high) have a unifrom DS around 0.5 so no split

In [ ]:
fp_ffr = torch.where((test_pred_ffr == 1) & (test_y == 0))[0]

f, ax = plt.subplots(1,3, figsize=(15,3))
ax[0].hist(test_ffr[fp_ffr], bins=30)
ax[0].set_title("FFR distribution false pos")
ax[1].hist(test_ds[fp_ffr], bins=20)
ax[1].set_title("DS distribution for FFR false pos preds")
ax[2].hist(test_ds[np.where((test_ffr[fp_ffr] >=0.7) & (test_ffr[fp_ffr] <0.8))[0]], bins=20)
ax[2].set_title("DS distribution for FFR false pos preds focus on FFR \in [0.7, 0.8)")


f.tight_layout()

In [ ]:
tp_ffr = torch.where((test_pred_ffr == 1) & (test_y == 1))[0]
fn_ds = torch.where((test_pred_ds == 0) & (test_y == 1))[0]

mask = np.intersect1d(tp_ffr, fn_ds)

f, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].hist(test_ffr[mask])
ax[0].set_title("FFR true positive distribution (intersect DS fn)")
ax[1].hist(test_ds[mask], bins=20)
ax[1].set_title("FFR true positive's DS distribution (intersect DS fn)")

* check the data

In [ ]:
df.columns

In [ ]:
import seaborn as sns

f, ax = plt.subplots(2,2, figsize=(10,10))
sns.histplot(x=df[df.has_lesion].DM_Insulin.values, hue=df[df.has_lesion].VOCE.values, ax=ax[0][0])
ax[0][0].set_title("Insulin threshold (hue: VOCE)")

sns.histplot(x=df[df.has_lesion].Prev_MI.values, hue=df[df.has_lesion].VOCE.values, ax=ax[0][1])
ax[0][1].set_title("Prev_MI (hue: VOCE)")

sns.histplot(x=df[df.has_lesion].Lesion_Length.values, hue=df[df.has_lesion].VOCE.values, ax=ax[1][0])
ax[1][0].set_title("Lesion_Length (hue: VOCE)")
ax[1][0].set_xlim(0,40)

sns.histplot(x=df[df.has_lesion].Smoker.values, hue=df[df.has_lesion].VOCE.values, ax=ax[1][1])
ax[1][1].set_title("Smoker (hue: VOCE)")

# Transformations on data

* standard scaling 

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from cardio.dataset.fame2_raw_data_loader import Fame2RawDSLoader, load_train_test_split


df = pd.read_csv("data/fame2_clinical_events_2year_data.csv")
df.head()

CLINICAL_DATA_FEATURES = ['FFR', 'DS', 'HTN', 'Hchol', 'DM_Overall',
       'DM_Insulin', 'Ren_Ins', 'PVD', 'CVA', 'Prev_MI', 'Prev_PCI', 'CCS',
       'CCS_3', 'Silent_Ischemia', 'LVEF', 'MVD', 'Lesion_Type', 'Age', 'Male',
       'BMI', 'CAD', 'Smoker', 'Age_64', 'BMI_28']

train_pids, test_pids = load_train_test_split("./data/datasets/simpleSplit")
train_df = df[(df.Patient.isin(train_pids))]


scaler = StandardScaler().fit(train_df[CLINICAL_DATA_FEATURES])

In [ ]:
df

In [ ]:
df[CLINICAL_DATA_FEATURES] = scaler.transform(df[CLINICAL_DATA_FEATURES])

df.to_csv("data/fame2_clinical_events_2year_data_std_scaled.csv")

# Add radiomix slices in GNN


In [ ]:
import cardio.resolvers as resolvers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cardio.dataset import SplitType
from cardio.configuration import *
from cardio.dataset import Fame2RawDSLoader
from cardio.dataset.fame2_raw_data_loader import load_train_test_split

fame2_ds = Fame2RawDSLoader(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    generate_data_with_2_views=False,
    duplicate_imgs_with_multiple_lesions=False,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup()

In [ ]:
from radiomics import firstorder, shape2D, glcm, featureextractor
import SimpleITK as sitk
import cv2 
from skimage.morphology import skeletonize
import logging

# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)

def gen_radius_mask_2d(array, i,j,radius, shape_mask=None):
    ''' Creates a 2d mask like array, with 0 everywhere except (i,j) and a radius around it which has 1
        if shape_mask != None then do a bitwise and between the radius mask and the lesion mask
    '''
    mask = np.zeros_like(array)
    mask[i,j] = 1
    d1l = i-radius if i-radius >= 0 else 0 
    d2l = j-radius if j-radius >= 0 else 0 
    mask[d1l:i+(radius+1), d2l:j+(radius+1)] = 1

    if isinstance(shape_mask, np.ndarray):
        return np.bitwise_and(mask, shape_mask)
    else:
        return mask

def generate_radiomix_per_coordinate(radiomix_extractor, nb_features, coordinates, raw_img, artery_mask, radius=5):
    ''' Generate radiomix features for each coordinate pair

    '''
    feature_values = np.zeros((len(coordinates), nb_features), dtype=np.float32)
    features = None
    for idx, (i,j) in enumerate(coordinates):
        ma = gen_radius_mask_2d(raw_img, i, j, radius, artery_mask)

        im = sitk.GetImageFromArray(raw_img)
        ma = sitk.GetImageFromArray(ma)

        featureVector = radiomix_extractor.execute(im, ma)
        if features == None:
            features = [k[9:] for k in featureVector.keys() if 'original' == k[:8] ]
        feature_values[idx,:] = np.array([featureVector['original_'+k] for k in features])

    return features, feature_values


* Create dataset

In [ ]:
from tqdm import tqdm
import pickle

radiomix_categories = ['glcm', 'glrlm', 'shape2D']
radiomix_shape2D_features = ['MeshSurface', 'Perimeter', 'MaximumDiameter', 'MajorAxisLength', 'MinorAxisLength', 'Elongation']

# The features generated by this categories are 46, please dont change anything
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.disableAllFeatures()
extractor.enableFeaturesByName(shape2D=radiomix_shape2D_features)
extractor.enableFeatureClassByName('glcm')
extractor.enableFeatureClassByName('glrlm')

# Stores all data in a list with format: (base_path, patient_id, lesions_id_syntax, radiomix_features)
data = [ ]
for ds_item in tqdm(fame2_ds):
    # Other features
    base_path  = ds_item[2].base_path.values[0]
    patient_id = ds_item[2].patient_id_x.values[0]
    lesion_id_syntax =  ds_item[2].lesion_id_syntax.values[0]
    VOCE =  ds_item[2].Event.values[0]

    raw_stacked = ds_item[0]
    lesion_mask = ds_item[1]
    raw_img = raw_stacked[0].numpy().astype(np.uint8)
    artery_mask = raw_stacked[1].numpy().astype(np.uint8)
    lesion_mask = lesion_mask.numpy().astype(np.uint8)

    # Fill the mask
    kernelSize = (4,4)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
    artery_mask_repaired = cv2.morphologyEx(artery_mask, cv2.MORPH_CLOSE, kernel)

    # centerline mask
    centerline_mask = skeletonize(artery_mask_repaired).astype(np.uint8)
    coordinates = np.where(centerline_mask != 0)
    coordinates = np.vstack(coordinates).transpose(1,0)

    # randomly sparcify centerline
    nb_choice = int(np.ceil(len(coordinates) * 0.5))
    indices = np.random.choice(coordinates.shape[0], nb_choice, replace=False)
    sparse_centerline  = np.zeros_like(centerline_mask)
    sparse_centerline[coordinates[indices][:,0], coordinates[indices][:,1]] = 1
    coordinates = np.where(sparse_centerline != 0)
    coordinates = np.vstack(coordinates).transpose(1,0)

    # generate radiomix features
    try:
        features, values = generate_radiomix_per_coordinate(extractor, 46, coordinates, raw_img, artery_mask_repaired, 15)
        row = (base_path, patient_id, lesion_id_syntax, VOCE, values)
        data.append(row)
    except a as Exception:
        print("skipped item with base_path: ", base_path)
        print(a)

pickle_out = open("data/datasets/radiomixOnCenterline/radiomix_data.pkl","wb")
pickle.dump(data, pickle_out)
pickle_out.close()

# Try to create patches from image

In [ ]:
# PLOTS

# masks = []
# for i,j in coordinates:
#     ma = gen_radius_mask_2d(raw_img, i, j, 10, artery_mask)
#     masks.append(ma)


f, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(2 * centerline_mask + lesion_mask)

# m = np.zeros_like(masks[0])
# for ma in masks: m += ma
ax[1].imshow(m)
# ax[1].imshow(2 * sparse_centerline + lesion_mask)

In [ ]:
import cv2 
from skimage.morphology import skeletonize

ds_item = fame2_ds[0]

raw_stacked = ds_item[0]
lesion_mask = ds_item[1]
raw_img = raw_stacked[0].numpy().astype(np.uint8)
artery_mask = raw_stacked[1].numpy().astype(np.uint8)
lesion_mask = lesion_mask.numpy().astype(np.uint8)

# Fill holes in the mask
kernelSize = (4,4)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
artery_mask_repaired = cv2.morphologyEx(artery_mask, cv2.MORPH_CLOSE, kernel)

# centerline mask
centerline_mask = skeletonize(artery_mask_repaired).astype(np.uint8)
coordinates = np.where(centerline_mask != 0)
coordinates = np.vstack(coordinates).transpose(1,0)

coordinates

In [ ]:
# Create an algo that will iterate coordinates 
# an start eliminating all of them that are in a radius r
# Then it will jump in the closest non eliminated coordinate and do the same
from sklearn.metrics import pairwise_distances



def erase_radius(mask, i, j, radius):
    assert mask[i,j] == 1
    d1l = i-radius if i-radius >= 0 else 0 
    d2l = j-radius if j-radius >= 0 else 0 
    mask[d1l:i+(radius+1), d2l:j+(radius+1)] = 0
    mask[i,j] = 1

def refresh_coordinates(mask):
    coordinates = np.where(mask != 0)
    coordinates = np.vstack(coordinates).transpose(1,0)
    return coordinates



f,ax = plt.subplots(1,2)

# find the start
mask = np.copy(centerline_mask)
stack = list()
r = 10

# Start the loop
ax[0].imshow(mask)

i,j = coordinates[0,0], coordinates[0,1]
erase_radius(mask, i, j, r)

ax[1].imshow(mask)
ax[1].scatter([j], [i], s=5)

coordinates = refresh_coordinates(mask)

pairwise_distances(coordinates, coordinates[0].reshape(1,-1)).shape

In [ ]:
a = pairwise_distances(coordinates, coordinates[0].reshape(1,-1))

np.linalg.norm(coordinates - coordinates[0], axis=1)

# Visual Transformers using patches

In [ ]:
import cardio.resolvers as resolvers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cardio.dataset import SplitType
from cardio.configuration import *
from cardio.dataset import Fame2RawDSLoader
from cardio.dataset.fame2_raw_data_loader import load_train_test_split
from tqdm import tqdm
import pickle

fame2_ds = Fame2RawDSLoader(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    generate_data_with_2_views=False,
    duplicate_imgs_with_multiple_lesions=False,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup()

* util functions

In [ ]:
import matplotlib.cm as cm
import random

def extract_patches(img, img_mask, patch_size, nb_patch, rnd=0, debug=False):
    patches = []
    
    img_shape = img.shape
    
    # If the patch is too big, extract a smaller one
    reduction_ratio = 1
    while patch_size >= img_shape[0]//2 or patch_size >= img_shape[1]//2:
        patch_size //= 2
        reduction_ratio *= 2
    
    # Do not extract on the border
    img_mask[:, :int(patch_size/2)] = 0
    img_mask[:int(patch_size/2), :] = 0
    img_mask[int(-patch_size/2):, :] = 0
    img_mask[:, int(-patch_size/2):] = 0
    
    # Select uniformly (+ some noise) the position of the extracted patches
    x_line = np.linspace(start=patch_size//2+1, stop=img_shape[1]-patch_size-1, num=nb_patch)
    x_line += (patch_size//2-1)*(np.random.rand(nb_patch))
    
    if debug:
        fig, ax = plt.subplots()
        ax.imshow(img, cmap=cm.gray)
        proba_dist_plt = np.copy(img_mask)
        proba_dist_plt = 255*proba_dist_plt/np.max(proba_dist_plt)
        proba_dist_plt[proba_dist_plt>0] = 255
        ax.imshow(proba_dist_plt, cmap=cm.gray, interpolation='none', alpha=0.5)
        ax.set_title("Proba distribution")
            
    for i in range(0, nb_patch):
        width_pos = int(x_line[i])
        proba_line = img_mask[:,width_pos].numpy() # for some reason choice does not work with tensor
        
        if random.random() < rnd:
            height_pos = patch_size//2+1+int((img_shape[0]-patch_size-1)*random.random())
        else:
            if np.sum(proba_line)  == 0:
                height_pos = patch_size//2+1+int((img_shape[0]-patch_size-1)*random.random())
            else:
                proba_line /= np.sum(proba_line)
                height_pos = np.random.choice(img_shape[0], p=proba_line)

        if debug:
            plt.plot(width_pos, height_pos, 'ro')
            plt.vlines(width_pos, 0, img_shape[0]-1, colors="red", linestyles="--")

        extracted_patch = img[int(height_pos-patch_size/2):int(height_pos+patch_size/2), 
                                        int(width_pos-patch_size/2):int(width_pos+patch_size/2)]
        
        if reduction_ratio != 1:
            resizer = transforms.Resize(extracted_patch.shape[0]*reduction_ratio)
            extracted_patch = resizer(extracted_patch.unsqueeze(0).unsqueeze(0))[0,0,:,:]
        
        patches.append(extracted_patch)
        
    if debug:
        plt.title("Patches center position")
        plt.show()
        
    if debug:
        fig, axs = plt.subplots(1, nb_patch)
        for i, patch in enumerate(patches):
            axs[i].imshow(patch, cmap="gray")
            axs[i].set_axis_off()
        fig.tight_layout()
        fig.show()
        plt.show()
        
    return patches


* Create dataset

In [ ]:
# Stores all data in a list with format: (base_path, patient_id, lesions_id_syntax, radiomix_features)
data = [ ]
for ds_item in tqdm(fame2_ds):
    # Other features
    base_path  = ds_item[2].base_path.values[0]
    patient_id = ds_item[2].patient_id_x.values[0]
    lesion_id_syntax =  ds_item[2].lesion_id_syntax.values[0]
    VOCE =  ds_item[2].Event.values[0]

    raw_stacked = ds_item[0]
    lesion_mask = ds_item[1]
    raw_img = raw_stacked[0].numpy().astype(np.uint8)
    artery_mask = raw_stacked[1].numpy().astype(np.uint8)
    lesion_mask = lesion_mask.numpy().astype(np.uint8)

    # Fill the mask
    kernelSize = (4,4)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
    artery_mask_repaired = cv2.morphologyEx(artery_mask, cv2.MORPH_CLOSE, kernel)

    # centerline mask
    centerline_mask = skeletonize(artery_mask_repaired).astype(np.uint8)

    # generate radiomix features
    try:
        # We need 9 patches of 64 so when stacking them up to 
        # create 1 image that will be fed to ViT
        patches = extract_patches(
            raw_img, torch.tensor(centerline_mask, dtype=torch.float64), 
            64, 9, rnd=0.0, debug=False
        )
        # Stack patches in 1 image ready to be passed to ViT
        img = torch.tensor(np.vstack([
            np.hstack(patches[:3]),
            np.hstack(patches[3:6]),
            np.hstack(patches[6:9])
        ])).unsqueeze(0).to(torch.float32)

        row = (base_path, patient_id, lesion_id_syntax, VOCE, img)
        data.append(row)
    except a as Exception:
        print("skipped item with base_path: ", base_path)
        print(a)

pickle_out = open("data/datasets/patchesForVitOnCenterline/data.pkl","wb")
pickle.dump(data, pickle_out)
pickle_out.close()


# Test stuff


In [ ]:
from cardio.pylightning import Fame2GraphSimpleDataModule

class Confs:
    dataset_dir = 'data/datasets/forecasting/knnOnlyLesionsFull'
    train_val_split = 'data/datasets/val_train_split'
    standardize_graph=False
    standardize_global=True
    skip_pixel_intensity=False
    use_balanced_batches=True
    num_workers=4
    train_batch_size=2
    m = 0
    v = 1
    gnn_cls_name='GIN'
    gnn_pooling_cls_name="CustomPooling"
    gnn_node_features=4
    gnn_node_emb_dim=512
    gnn_nb_layers=3
    gnn_dropout=0.5
    gnn_norm="BatchNorm"
    gnn_act="ReLU"
    gnn_ffr_pooling_factor=2
    gnn_ffr_pooling_proj_dim=16
    gnn_global_hidden_dim=512
    nb_classes=1

    use_lesion_wide_info=True
    pool_only_lesion_points=False
    node_pooling="all_stats"



confs = Confs()

dm = Fame2GraphSimpleDataModule(confs)
dm.setup(None)
dl = iter(dm.train_dataloader())
b = next(dl)
print("len of batch", len(b))


In [ ]:
import cardio.networks.gnn as gnn


net = gnn.GNNClassifier(
        gnn.gnn_resolver(confs),
        gnn.ConfigurablePooling(confs), 
        confs
)

net(b)

In [ ]:
from cardio.dataset import *
import os.path as op

graphDsWrapper = Fame2GraphDatasetWrapper()
train_ds = graphDsWrapper.instantiate_Fame2GraphDataset(None, None, root=op.join("data/datasets/forecasting/knn5Inner05OnlyLesionsFull", "train"))

In [ ]:
import numpy as np

np.mean([[1,0],[0,2], [1,1]], axis=1)

In [ ]:
import torch 

x = train_ds[0].x
edge_index = train_ds[0].edge_index.transpose(1,0)
coord = torch.vstack([x[:,0], x[:,1]]).transpose(1,0)

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt

def create_graph(coordinates, edge_index):
    g = ig.Graph(len(coordinates))
    g.vs['x'] = coordinates[:,0].tolist()
    g.vs['y'] = coordinates[:,1].tolist()

    coords = g.layout_auto().coords
    for edge in edge_index:
        g.add_edge(edge[0], edge[1])
    g.simplify() 

    return g

f,a = plt.subplots()
g = create_graph(coord, edge_index)
ig.plot(
    g,
    target=a,
    vertex_size=0.04,
    vertex_color="lightblue",
    edge_width=0.8
)

In [ ]:
from cardio.networks import baseline

class Confs:
    cnn_dropout = 0.5
    weight_init_strategy="Xavier Uniform"
net = baseline.BaselineResNet( Confs())

In [ ]:
net

# Create Cnn2Gnn dataset

In [ ]:
import cardio.resolvers as resolvers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cardio.dataset import SplitType
from cardio.configuration import *
from cardio.dataset import Fame2RawDSLoader
from cardio.dataset.fame2_raw_data_loader import load_train_test_split

fame2_ds = Fame2RawDSLoader(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    generate_data_with_2_views=False,
    duplicate_imgs_with_multiple_lesions=True,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

fame2_ds.setup()

In [ ]:
class DelaunayVertexConnector:
    def __init__(self, distance_cuttoff) -> None:
        super().__init__()
        self.distance_cutoff = distance_cuttoff
        self.store_pruned_simplices = []

    def connect(self, coordinates: torch.Tensor, features: torch.Tensor) -> torch.Tensor:
        triangulation = Delaunay(coordinates)
        return self.to_edges_coo(self.prune_simplices(triangulation, coordinates))

    def prune_simplices(self, triangulation, points):
        idxes_to_keep = []
        cnt = 0
        cond = lambda di: di > self.distance_cutoff
        for i, row in enumerate(points[triangulation.simplices]):
            bitmap = [0,0,0]
            bitmap[0] = cond(torch.norm((row[0]- row[1]).to(torch.float)))
            bitmap[1] = cond(torch.norm((row[0]- row[2]).to(torch.float)))
            bitmap[2] = cond(torch.norm((row[2]- row[1]).to(torch.float)))
            if sum(bitmap) != 0: 
                cnt
                continue
            idxes_to_keep.append(i)
        self.store_pruned_simplices.append(cnt)
        return triangulation.simplices[idxes_to_keep]

    def to_edges_coo(self, simplices):
        edge_indexes = set()
        for tri in simplices:
            edge_indexes.add((tri[0], tri[1]))
            edge_indexes.add((tri[1], tri[0]))
            edge_indexes.add((tri[0], tri[2]))
            edge_indexes.add((tri[2], tri[0]))
            edge_indexes.add((tri[1], tri[2]))
            edge_indexes.add((tri[2], tri[1]))
        return torch.LongTensor(list(edge_indexes))

In [ ]:
import pickle
from tqdm import tqdm
import numpy as np

# Create the dataset
conn = DelaunayVertexConnector(10)

data = []
for ds_item in tqdm(fame2_ds):
    # Other features
    base_path  = ds_item[2].base_path.values[0]
    patient_id = ds_item[2].patient_id_x.values[0]
    lesion_id_syntax =  ds_item[2].lesion_id_syntax.values[0]
    VOCE =  ds_item[2].Event.values[0]

    img = ds_item[0][0]
    mask = ds_item[0][1].to(torch.bool)
    coordinates = torch.vstack(torch.where(mask == True))
    edge_index = conn.connect(coordinates.transpose(1,0), None)

    data.append((img, mask, edge_index, base_path, patient_id, lesion_id_syntax, VOCE ))


pickle_out = open("data/datasets/eventForecastingCnn2Gnn/data.pkl","wb")
pickle.dump(data, pickle_out)
pickle_out.close()

print(np.mean(conn.store_pruned_simplices))

* test cnn2gnn

In [ ]:
from cardio.dataset import Cnn2GnnDataset, SplitType
from cardio.pylightning.dm_fame2_graph import Fame2GraphKFoldDataModule

class Confs:
    dataset_dir = "data/datasets/lesionDetection/Delaunay5_Inner050_Alwayscenter"
    use_balanced_batches = "True"
    num_workers = 4
    train_batch_size = 2
    test_batch_size=2
    standardize_graph=True
    standardize_global=True
    device="cpu"
    use_lesion_wide_info=True
    skip_pixel_intensity=True
    

dm = Fame2GraphKFoldDataModule(Confs())
dm.setup(None)
dm.setup_folds(5)
dm.setup_fold_index(0)
ds = dm.train_dataset

In [ ]:
ds = Cnn2GnnDataset("data/datasets/eventForecastingCnn2Gnn", SplitType.train, split_path="data/datasets/eventForecastingCnn2Gnn")

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ds[100][0].flatten(), bins = 60)
plt.show()

In [ ]:
import numpy as np
idx = np.random.choice(len(ds), int(len(ds)*0.2))
ds_v = ds.index_select(idx, True)

In [ ]:
from cardio.networks.cnn2gnn import Cnn2Gnn
class Confs:
    gnn_node_emb_dim=512
    gnn_nb_layers = 3
    gnn_node_features = 32
    nb_classes=1
    gnn_dropout = 0.5
    gnn_cls_name = "CustomGIN"

    cnn_out_channels = 64
    cnn_kernel = 2

b = next(iter(dm.train_dataloader()))
net = Cnn2Gnn(Confs())
net(b[0])

# Create the CNN dataset again

In [ ]:
import cardio.resolvers as resolvers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cardio.dataset import SplitType
from cardio.configuration import *
from cardio.dataset import Fame2RawDSLoader
from cardio.dataset.fame2_raw_data_loader import load_train_test_split

fame2_ds = Fame2RawDSLoader(
    FAME2_DUMP_DIR, 
    FILEPATH_CLINICAL_EVENT_DF, 
    only_lesion_data_points=True,
    generate_data_with_2_views=False,
    duplicate_imgs_with_multiple_lesions=True,
    df_ce_event_columns=EVENT_COLUMNS,
    generate_lesion_labels=True, 
    lazy_load=True,
    all_arteries_in_one_img=False)

In [ ]:
fame2_ds.df.patient_id_x.unique().shape

In [ ]:
113/562

In [ ]:
450 + 113

In [ ]:
fame2_ds.setup("data/datasets/simpleSplit/", SplitType.train)

In [ ]:
fame2_ds.setup("data/datasets/simpleSplit/", SplitType.test)

In [ ]:
fame2_ds.setup()

In [ ]:
from cardio.dataset.preprocess import graph
from tqdm import tqdm
from cardio.dataset.fame2_patch_dataset import create_ds_split

save_dir = "data/datasets/cnn/patches_200x200_GlobFeat"

features = ["DS", "FFR", "Lesion_Length",
            "Lesion_Type", "artery", "exactSegmentLocation"]
feature_transforms = [None, None, None,
    graph.lesion_type_to_1hot,
    graph.artery_to_1hot,
    graph.artery_segment_to_float
]

# Generate the dataset
fame2_ds.setup(SPLIT_PATH, SplitType.train)
index, data = zip(*fame2_ds.data)
df = pd.concat([d for d in data])
df.index = index
create_ds_split(df, True, features, feature_transforms, save_dir)

fame2_ds.setup(SPLIT_PATH, SplitType.test)
index, data = zip(*fame2_ds.data)
df = pd.concat([d for d in data])
df.index = index
create_ds_split(df, False, features, feature_transforms, save_dir)

In [ ]:
# test the lesion wide dataset
from cardio.dataset.fame2_patch_dataset import Fame2CNNDataset
from cardio.dataset import SplitType

ds = Fame2CNNDataset("data/datasets/cnn/patches_200x200_GlobFeat", SplitType.test, None)


In [ ]:
import matplotlib.pyplot as plt

i=200
f,ax = plt.subplots(1,2)
ax[0].imshow(ds[i][0][0][0,:])
ax[1].imshow(ds[i][0][0][1,:])

In [ ]:
from cardio.pylightning.dm_fame2_baseline import Fame2BaselineKFold 

class Confs:
    n = 0
    dataset_dir="data/datasets/cnn/patches_200x200_GlobFeat"
    num_workers = 4
    train_batch_size = 2
    standardize_global=True
    use_lesion_wide_info = True
    use_balanced_batches = True
    standardize_img = True
    skip_pixel_intensity = True
    weight_init_strategy='pre_trained'
    device = "cpu"
    

confs = Confs()
dm = Fame2BaselineKFold(confs)
dm.setup(None)
dm.setup_folds(5)
dm.setup_fold_index(0)

In [ ]:
from cardio.pylightning.dm_fame2_graph import Fame2GraphSimpleDataModule 

class Confs:
    n = 0
    dataset_dir="data/datasets/forecasting/Delaunay5_Inner050_Alwayscenter_OnlyLesions_FUL/"
    num_workers = 4
    train_batch_size = 2
    standardize_global=True
    use_lesion_wide_info = True
    use_balanced_batches = True
    standardize_graph = True
    skip_pixel_intensity = True
    dont_validate=True
    device = "cpu"
    

confs = Confs()
dm = Fame2GraphSimpleDataModule(confs)
dm.setup(None)
# dm.setup_folds(5)
# dm.setup_fold_index(0)

In [ ]:
import numpy as np
n = np.arange(10)

n

In [ ]:
i = next(iter(dm.train_dataloader()))

In [ ]:
from cardio.networks.cnn_baseline import BaselineResNet
from cardio.resolvers import  gnn_resolver
%load_ext autoreload
%autoreload 2

confs.gnn_cls_name = "GIN"
confs.gnn_node_emb_dim = 256
confs.gnn_nb_layers = 5
confs.gnn_act = 'ReLU'
confs.gnn_norm = "BatchNorm"
confs.gnn_pooling_cls_name = "ConfigurablePooling"
confs.pool_only_lesion_points = False
confs.gnn_is_siamese = False
confs.node_pooling = "all_stats"
confs.gnn_use_global_info = True
confs.gnn_dropout = 0.2
confs.nb_classes = 1
confs.cnn_dropout = 0.5
net = gnn_resolver(confs)

net

In [ ]:
net(i)